# Luangwa Basin Time Series Analysis for MSC IWRM

## Preliminary processing of data
Time is the most important thing in a time-series. Therefore it is important to convert the data into the appropriate format

### 1. Importing packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.graphics.tsaplots as sgt
import statsmodels.tsa.stattools as sts
from statsmodels.tsa.seasonal import seasonal_decompose
import seaborn as sns
sns.set()


### 2. Importing Data

In [2]:
raw_csv_data = pd.read_csv('2013-18MOD16A2Station_Serenje2.csv')
df_comp = raw_csv_data.copy()

### 3. Viewing Data

In [3]:
df_comp.head(24)

,date,Satellite_ET,ScaledS_ET,Station_ET
0,1/1/2013,263.876,26.388,NaN
1,2/1/2013,324.463,32.446,NaN
2,3/1/2013,355.000,35.500,NaN
3,4/1/2013,262.270,26.227,NaN
4,5/1/2013,188.725,18.873,NaN
5,6/1/2013,128.474,12.847,NaN
6,7/1/2013,108.482,10.848,NaN
7,8/1/2013,74.753,7.475,NaN
8,9/1/2013,65.583,6.558,NaN
9,10/1/2013,107.078,10.708,-4.102158


### 4. Getting information about data

#### a. Check the data type and check the data information

In [4]:
df_comp.describe()

,Satellite_ET,ScaledS_ET,Station_ET
count,72.000000,72.000000,53.000000
mean,188.649889,18.865028,40.389501
std,88.194543,8.819452,30.630458
min,64.361000,6.436000,-16.195257
25%,108.131000,10.813000,17.128312
50%,174.077000,17.407500,40.665133
75%,264.219250,26.422250,70.002893
max,363.137000,36.314000,81.476488


When we zoom in on the date (below)If you notice, the "top" value is not the highest frequency... All values in python are 1s as they are not equal to 0. There for any single value holds a "top" (1) value! The system randomly selects any value. We must convert this column "Date" into a datetime type. 

In [5]:
df_comp.date.describe()

count           72
unique          72
top       1/1/2013
freq             1
Name: date, dtype: object

#### b. Check the data type

In [6]:
df_comp.dtypes

date             object
Satellite_ET    float64
ScaledS_ET      float64
Station_ET      float64
dtype: object

#### c. Check the data information

In [7]:
df_comp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          72 non-null     object 
 1   Satellite_ET  72 non-null     float64
 2   ScaledS_ET    72 non-null     float64
 3   Station_ET    53 non-null     float64
dtypes: float64(3), object(1)
memory usage: 2.4+ KB


### 5. Convert the "Date" column to 'datetime' for Pyrhon to understand that it is a date

In Pandas, we have a method (to.date) we can call to convert the "Date" values to date from text whatever format - in this case 'object'. Also in our case, it is the entire data frame or complete data frame (df_comp).

#### Date is being converted from object to date recognisable by Python

Call the method pd.to_datetime(), as arguement enter(call) the column in question - in this case "Date": pd.to_datetime(df_comp.Date). The systeme (Python) assumes a string in a "mm/dd/yyy" form is being plugged in. Remember most dates are saved in the "mm/dd/yyyy"format. To get around this, a second argument is used: dayfirst = True.  

In [8]:
pd.to_datetime(df_comp.date, dayfirst = True)

0    2013-01-01
1    2013-01-02
2    2013-01-03
3    2013-01-04
4    2013-01-05
        ...    
67   2018-01-08
68   2018-01-09
69   2018-01-10
70   2018-01-11
71   2018-01-12
Name: date, Length: 72, dtype: datetime64[ns]

The column will just display the changes without actually storing the data. To store it, we designate the changes to "df_comp.Date" in the first line as done below. 

In [9]:
df_comp.date = pd.to_datetime(df_comp.date, dayfirst = True)

To see what have done, we use the .head() to explore(view) the changes. See below! 😉 Here we don't see that the values are formatted differently because the dates were already in the yyyy-mm-dd format. 

In [10]:
df_comp.head()

,date,Satellite_ET,ScaledS_ET,Station_ET
0,2013-01-01,263.876,26.388,NaN
1,2013-01-02,324.463,32.446,NaN
2,2013-01-03,355.000,35.500,NaN
3,2013-01-04,262.270,26.227,NaN
4,2013-01-05,188.725,18.873,NaN


To see if the "Date" values are no longer stored as text. We can use the .describe() method to see. Equally, the .dtypes and info() methods can work. 

We only call the method on the attribute "Date"we want to see:As we do not want to see the other attributes, we specify the attribute we want to see (df_comp.Date)

In [11]:
df_comp.date.describe()

count                     72
mean     2015-07-08 08:00:00
min      2013-01-01 00:00:00
25%      2014-01-06 18:00:00
50%      2015-07-08 00:00:00
75%      2017-01-06 06:00:00
max      2018-01-12 00:00:00
Name: date, dtype: object

As you can see, now the data has been arranged with the first date on the top as it should be. 

In [12]:
df_comp.dtypes

date            datetime64[ns]
Satellite_ET           float64
ScaledS_ET             float64
Station_ET             float64
dtype: object

### 6. Set the "Date" column to be the index

We are setting the  date attribute to be the referred to index. The optional argument "inplace", tells Python to set this new format instead of integer. 

In [13]:
df_comp.set_index('date', inplace = True) 

In [14]:
df_comp.head() #We can see that the "Date" is now the index and that now it will not be recognised as an attribute. 

,Satellite_ET,ScaledS_ET,Station_ET
date,,,
2013-01-01,263.876,26.388,NaN
2013-01-02,324.463,32.446,NaN
2013-01-03,355.000,35.500,NaN
2013-01-04,262.270,26.227,NaN
2013-01-05,188.725,18.873,NaN


This time, when we try to call the date column using describe. We will see an error because the column is no longer an attribute but the index. See below

In [15]:
df_comp.date.describe()

AttributeError: 'DataFrame' object has no attribute 'date'

### 7. Time-Series Frequency

#### Time series data requires a constant frequency that persists through out the data set

#### a. Before we set the frequency, it is important to check for missing values

In [ ]:
df_comp.isna()

In [ ]:
df_comp.isna().sum()

Theren't any missing values for the satellite dataa prior to us setting th frequency. However, we have 19 missing values for the station data 

#### b. Set the frequency

The frequency is set by calling the 'as frequent' (asfreq) method

In [ ]:
df_comp = df_comp.asfreq('d') #The method asfreq takes alphabetical values d = day, b = business day, year = a (annual)

In [ ]:
df_comp.head()

The process of assgning frequency may generate new periods, which do not have values associated with them. In this study, the besty frequency is daily 'd' as the others do not suitably bring out the data as needed. The others include: business days 'b' (which are days minus weekends and public holidays; weeks 'w'; months 'm'; years 'a' (represented by 'a' because of the use of 'annually' in place of 'yearly'. 

### 7. Handling Missing Values

##### First thing: Check if assigning frequency increased the dates for which data is not available. There is a dedicated method: isna() of which in our case we call on the entire data frame
👉🏾We call this method on the entire data frame
👉🏾True indicates there are missing values
👉🏾False indicates there aren't. 

In [ ]:
df_comp.isna()

The bigger the dataset, the more difficult it be comes to spot missing entries. Therefore, we employ the method below:

#### a. Viewing all (the sum Σ οf) missing values ( the "not available" values)

In [ ]:
df_comp.isna().sum() #We call the sum method .sum() - without available information(empty brackets, no arguments) in order to sum up all the Booleans! 🔥

As we can see, the values were not altered: Satellite data with no missing data and station data with 19 missing data points. 

#### b. Filling in the "not available" data, fillna(). 

There are three methods of filling in missing data. Front fill (ffill), back fill (bfill) & fill with the same values e.g the mean, (or median). Usually filling values using the mean is a bad approach because there are underlying values. This method is only appropriate when the data fluctuates around the mean. 

In [ ]:
df_comp.Station_ET = df_comp.Station_ET.fillna(method = 'ffill')

In [ ]:
df_comp.isna().sum() #Check new missing values

We can see that from 19, the number has reduced to only 9 instances after front filling. This is due to the nature of the data which has no data in front. 

#### To assign a constant value, we do not need an argument but a number (value)

In this instance, we try the mean. The reason why the mean is not advised is explained in the earlier sections

In [ ]:
# df_comp.Station_ET  = df_comp.Station_ET.fillna(value = df_comp.Station_ET.mean())
#df_comp.ScaledS_ET  = df_comp.ScaledS_ET.fillna(method = 'ffill')


In [ ]:
# df_comp.isna().sum()

In [ ]:
# Using the mean method (()) clears all instances to 0. 

In [ ]:
# df_comp.ScaledS_ET  = df_comp.ScaledS_ET.fillna(method = 'bfill')

In [ ]:
# df_comp.isna().sum()

### Let's try to analyse the data for outliers

In [ ]:
df_comp.describe()

In [ ]:
df_comp.head(12)

In [ ]:
df_comp.tail(12)

In [ ]:
sns.distplot(df_comp['Station_ET'])
plt.savefig("Serenje Station Outliers Data.png")

In [ ]:
sns.distplot(df_comp['ScaledS_ET'])
plt.savefig("Serenje Station Outliers Data1.png")

To see the outliers clearly we use box plots by calling on our column of interest from the sns package

    boxplot(DATA FRAME [])

In [ ]:
sns.boxplot(df_comp['ScaledS_ET'])
plt.savefig("Serenje_Sat_Outliers_BoxPlot.png")

In [ ]:
sns.boxplot(df_comp['ScaledS_ET'])
plt.savefig("Serenje Station Outliers BoxPlot.png")

There seems to be not outliers but here is the method to remove outliers if there are there. 

#### Adding or Removing Columns (Just in case)

In time sereis we often end up analyzing one sequence by its own. In this case, we are going to narrow down to the Station_ET. 

👉🏾 We remove data for several reasons 1. load less data at a time (this may not be a problem as the data  is not much) 2. Clarity                 

##### Before we remove any data, we will create a column called MOD16A2 and assign it with the same values as ScaledS_ET

This column makes things much more convenient as it allows us, by changing a single line, to resuse the entire code to analyze to analyze a different time series. 

👇🏾 Here below is how it is created

In [ ]:
df_comp['MOD16A2'] = df_comp.ScaledS_ET

In [ ]:
df_comp.describe()

📌 Since it has the same values as ScaledS_ET, the summerized statistics for the two are identical. 

#### Deleting

Now to delete a specifi column from the data frame, we use the following method

In [ ]:
del df_comp['Satellite_ET'],
del df_comp['ScaledS_ET'],
del df_comp['Station_ET']

In [ ]:
df_comp.describe()

### Processing of data

#### 1. Splitting of data

#### For successful machine🤖 learning, we need to split the available data into two sets: 
👉🏾 A training set and a testing set. The goal is to have the option of feeding new information into the model and comparing its predictions to actual values. The closer the forecasts match our value, the better the model performs.

For many ML methods we shuffle the data before splitting it however TS however time series data relies in keeping the chronological order of values within a setk. This unfortunately, makes shuffling impossible.

Since we cannot shuffle, the training and the testing set should be uninterupted sequences of values. 

The training set should include data from the beginning of a set, up to a specific point in time while the testing set - the rest. 
👉🏾 The appropriate size of the training set is debatable, if it is too long, the model will fit the actual data too well and will perform poorly with the new data. 

👉🏾If it is too small, we won't be able to create a model accurate enough. 

🎼 In this study, an 80%:20% is used (which is reasonable). 

##### Use the "iloc" method (coming from the index location) which slices the data
👉🏾 We must know where the first set begins and where the second one begins 
👉🏾 In other words, we must determine the cutoff point point. 

To achieve this, we use the "len" function which returns the length of an object in Python. 
👉🏾 Let's define an integer (int) variable called size which defines how long the training set should be. As stated before, we want to use approximately 80% of the entire data set: the result will be the length of the training set. 

Use the code below:

In [ ]:
size = int(len(df_comp)*0.8)
# df, df_test = df_comp.iloc[:size], df_comp.iloc[size:]

After determining where the training should end, we should now use the "iloc"

Training set will be named "df". the "df" is short hand to save time as we will refer to it a lot. 
Testing set will be name "df_test".

We refer df up to the size value ( which as defined above is an integer taking 80% of the entire data frame👍🏾)

In [ ]:
df = df_comp.iloc[size]

In [ ]:
df_test = df_comp.iloc[size]

In [ ]:
df, df_test = df_comp.iloc[:size], df_comp.iloc[size:]

In [ ]:
df.tail()

In [ ]:
df_test.head()

## Lesson from code basic

Date Time Index and resampling


SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (3752675155.py, line 1)

## Temperature Analysis

### De-trending and modelling seasonal variation with Fourier Series

### T seasonal = a0 + Σiαisin(iω1t +θ) + Σiβicos(iω2t + φ)

#### Time series decomposition

Technique that splits a time series into several components, each representing an underlying pattern category

    yt = Tt + St + et

1. Trend: decreasing or increasing over time.
2. Seasonality: periodic signal.
3. Noise: variability in data that the model can't explain. 

In [ ]:
Denoised DAT series

Denoise to see trend and overall pick. 

Method usually used: Convolutions 

(f*g)(x) = 

## Main approaches for mathematical modeling

### White Noise

#### 🙋🏾‍♂️ What at is it? Why do we need it?
What noise is special type of TS, where the data doesn't follow a pattern. 

💡A recap of TS is that, data in the past, also persists in the future (training:test).
    👉🏾 In this case, since no pattern can be found, the data is unpredictable. 
    👉🏾 In order for a series to be considered as 'white noise', it must satisfy the following three conditions:
        ✔ μ   - the mean must be constant
        ✔ σ^2 - constant standard deviations
        ✔     - no autocorrelation in any period: This means that there is no clear relationship between past and present values. 

The first two ideas are straight forward, on the third, we need to try and iterate a little more. 

ρ = corr( xt, xt-1)

##### White Noise is a sequence of random data, whree every value has a time-period associated with it
👉🏾We can say it behaves sporadically, hence there is no prediction into the future. 

Dictionary 📖
ρ σ μ λ Σ φ Φ γ Γ 

In (financial) modelling, it is important to distinguish between white noise data and regular TS data. We can easily tell the two apart by comparing  their graphs. 

wn = np.random.normal() 

This will create an array of values in a 'Normal' distribution 

    [x1, x2, x3, ..., xn]
    X ~ N(μ, σ^2)

If we want this series to be compared to the actual sequence, we should set its mean and standard deviation to that of the actual sequence data set. 

    X ~ N(μMOD, σ^2MOD)


the location argument of the method loc, takes numbers of the average point of the distribution
    (loc = df.MOD16A2.mean(), scale = df.MOD16A2.std())

Before calling the method, we define how many values we want it to generate. If we want the sequence to serve as a good comparison, it should have 
as many elements (values) as our time series. 

so let's set the size argument as shown below:

  loc = df.MOD16A2.mean(), scale = df.MOD16A2.std(), size = len(df) then run the cell!

In [ ]:
wn = np.random.normal(loc = df.MOD16A2.mean(), scale = df.MOD16A2.std(), size = len(df))

#### We now add the White Noise to the dataframe

In [ ]:
df['White_Noise'] = wn

💡 Every value of the WN sequence will be assigned a time period since the df uses dates as indices 

☝🏾 The ⚠ message above will be  discussed

Let's how the data frame looks like after adding a new column

In [ ]:
df.describe()

##### 🚦 Notice that that the mean of the series (MOD16A2) is similar to that of the White Noise

This is because the White Noise is generated around the meanof MOD16A2. However, since each data set is generated separately, the mean cannot be the same. 

##### Let's now name and plot it. It is important ot name to avoid mistakes when extracting insights

It's important to note that the size method is assigned a number to make the size distinguishable. 

We also call the figure size (figsize) to stretch the graph and be able to observe the values clearly. figsize is an argument of the plot method plt()

In [ ]:
df.White_Noise.plot(figsize =(15, 3.5))
plt.title("White Noise Time-Series", size = 24)
plt.show()

##### We now plot the MOD16A2 data with similar criteria to make the two graphs comparable 
If the graphs are different (i.e the y axis), we can modify the respective graph to match the other one using this code: 
    plt.ylin(a, b) where a is the lower limit and b is the upper limit. In this case, everythin looks in order.

In [ ]:
df.MOD16A2.plot(figsize =(15, 3.5))
plt.title("MOD16A2-Satellite Evaporation", size = 24)
plt.show()

##### 💡Looking at the two graphs, the WN graph has more jumps towards the data showing the randomness while the actual MPD16A2 data has even spikes which 
indicate a pattern

Let's join the two graphs in a White Noise vs MOD16A2 graph, add a lengend and download this figure using the two lines of code below: 

plt.legend(loc = [.2, .2]);
plot.savefig("MOD16A2 and Noise comparison.png")

In [ ]:
df.White_Noise.plot(figsize =(15, 3.5))
df.MOD16A2.plot(figsize =(15, 3.5))
plt.title("White Noise vs MOD16A2", size = 24)
plt.show()
#plt.legend(loc = [.2, .2]);
plt.savefig("MOD16A2 and Noise comparison.png")

💡🚦 See the trend in the orange line and the randomness of the blue line

### Random walk

As special type of time-series, where values tend to persist over time and the differences between periods are simply white noise. 

Take the scenario below:

Pt = Pt-1 + εt, where P: prices εt: residuals ~ WN(μ,σ^2) - cannot be predicted

In [ ]:
# rw = 
# rw.date = pd.to_datetime(rw.date, dayfirst = True)
# rw.set_index('Date', inplace= True)
# rw = rw.asfreq('b')

In [ ]:
df[Random_Walk] = rw.Station_ET

In [ ]:
# Let's generate whitel noise [x1, x2, x3, ..., xn] X~N(μ, σ^2)
wn = np.random.normal(loc = df.Uscaled_Sat_ET.mean(), scale = df.Uscaled_Sat_ET.std(), size = len(df))

In [ ]:
# Try adding the white noise to the data frame
df['wn'] = wn

In [ ]:
df.describe()

#White Noise

In [ ]:
df.wn.plot(figsize = (20,5))
plt.title('White Noise Time-Series', size = 24)
plt.show()
plt.savefig("White Noise Time-Series.png", dpi = 400)

#### saving the figure

In [ ]:
# plt.savefig("White Noise Time-Series.png", dpi = 400)

In [ ]:
df.Uscaled_Sat_ET.plot(figsize = (20, 5))
plt.title('Evaporation Trend*', size =24)
plt.show()

In [ ]:
df.wn.plot(figsize = (20,5))
df.Uscaled_Sat_ET.plot(figsize = (20, 5))
plt.title('Evaporation Trend* v White Noise', size =24)
plt.show()

In [ ]:
# # Pt = Pt-1 + εt, P: prics εt: residuals = WN(μ,σ^2) - cannot be predicted
# rw = pd.read_csv('RandWalk.csv')
# rw.date = pd.to_datetime(rw.date, dayfirst = True)
# rw.set_index('Date', inplace= True)
# rw = rw.asfreq('b')

In [ ]:
# Seasonality

In [ ]:
# Certain trends will appear in a cyclical manner
# Decompose the Time Series in to three effects
s_dec_additive = seasonal_decompose(df.Uscaled_Sat_ET, model = "additive")
s_dec_additive.plot()
plt.show()

In [ ]:
# Certain trends will appear in a cyclical manner
# Decompose the Time Series in to three effects
s_dec_additive = seasonal_decompose(df.Uscaled_Sat_ET, model = "multiplicative")
s_dec_additive.plot()
plt.show()

In [ ]:
#The trend is similar to  the observed series because: because the decomposition function uses the previous period values as a trend setter
#Values oscillating back and forth and the (scale) of the figure is too small to observe this change.No cyclic pattern determined    
# turn of the century and 2008. .com babble and housing. 